# Face Detection with Detectron2

<img src="https://dl.fbaipublicfiles.com/detectron2/Detectron2-Logo-Horz.png" width="500">

## 1. Install detectron2

In [ ]:
# install dependencies: 
!pip install pyyaml==5.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

In [ ]:
# install detectron2: (Colab has CUDA 10.1 + torch 1.8)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
import torch
assert torch.__version__.startswith("1.8")   # need to manually install torch 1.8 if Colab changes its default version
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

## 2. Import drive to use custom data (colab)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 3. Import modules

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

## 4. Load dataset (json)

In [ ]:
PATH = '/content/drive/MyDrive/Colab_Notebooks/dataset/od_custom/'

In [ ]:
import json

train_dataset_path = PATH + 'train/'
with open(train_dataset_path + 'via_region_data.json', 'r') as f:
  train_data = json.load(f)

In [ ]:
train_dataset_lst = [
  {
    'file_name': train_dataset_path + i['filename'],
    # 'height': 256, 'width': 256,
    'image_id': k,
    'annotations': [{
      'bbox_mode': detectron2.structures.BoxMode.XYWH_ABS,
      'bbox': [r['shape_attributes']['x'], r['shape_attributes']['y'], r['shape_attributes']['width'] - r['shape_attributes']['x'], r['shape_attributes']['height'] - r['shape_attributes']['y']],
      'category_id': 0
    } for r in i['regions']]
  }
  for k, i in train_data.items() if i['regions']
]

len(train_dataset_lst)

data example

In [ ]:
train_dataset_lst[0]

In [ ]:
val_dataset_path = PATH + 'val/'
with open(val_dataset_path + '/via_region_data.json', 'r') as f:
  test_data = json.load(f)

In [ ]:
test_dataset_lst = [
  {
    'file_name': val_dataset_path + i['filename'],
    # 'height': 256, 'width':256,
    'image_id': k,
    'annotations': [{
      'bbox_mode': detectron2.structures.BoxMode.XYWH_ABS,
      'bbox': [r['shape_attributes']['x'], r['shape_attributes']['y'], r['shape_attributes']['width'] - r['shape_attributes']['x'], r['shape_attributes']['height'] - r['shape_attributes']['y']],
      'category_id': 0
    } for r in i['regions']]
  }
  for k, i in test_data.items() if i['regions']
]

len(test_dataset_lst)

In [ ]:
train_dataset = train_dataset_lst

DatasetCatalog.clear()
MetadataCatalog.clear()

DatasetCatalog.register("face_train", lambda: train_dataset)
MetadataCatalog.get("face_train").set(thing_classes=['face'])

## 5. Visualizing dataset (train)

In [ ]:
for i, data in enumerate(train_dataset[:2]):
    img = cv2.imread(data["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=MetadataCatalog.get("face_train"), scale=1)

    start_point =(data['annotations'][0]['bbox'][0], data['annotations'][0]['bbox'][1])
    end_point =(data['annotations'][0]['bbox'][2], data['annotations'][0]['bbox'][3])

    out = visualizer.draw_dataset_dict(data)

    cv2.rectangle(out.get_image()[:, :, ::-1], start_point, end_point, (200,0,0), 2)
    cv2_imshow(out.get_image()[:, :, ::-1])
    print(i+1, data["file_name"])

## 6. Set Model, Hyperparameter & Train model

In [ ]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()

cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("face_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 6
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 6
cfg.SOLVER.BASE_LR = 0.01  # pick a good LR
cfg.SOLVER.MAX_ITER = 500
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (face)

# cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8

model_path = PATH + "models/faster_rcnn_r101_fpn"
cfg.OUTPUT_DIR = model_path
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

## 7.Load model to test

In [ ]:
# cfg already contains everything we've set previously. Now we changed it a little bit for inference:
cfg = get_cfg()

cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.DATALOADER.NUM_WORKERS = 20
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (face)

# cfg.MODEL.DEVICE='cpu'
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8

model_path = PATH + 'models/faster_rcnn_r101_fpn'
cfg.OUTPUT_DIR = model_path
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
predictor = DefaultPredictor(cfg)

## 8. Test with test another dataset

In [ ]:
test_dataset = [d for d in test_dataset_lst]

pred = []
real = []

#for i, d in random.sample(list(enumerate(te)), 10):
for i, data in enumerate(test_dataset[::1]):
    img = cv2.imread(data["file_name"])
    outputs = predictor(img)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    v = Visualizer(img[:, :, ::-1], metadata=MetadataCatalog.get("face_train"), scale=1)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])
    print(i+1, data["file_name"])

    pred.append(len(data['annotations']))
    real.append(np.array(outputs["instances"].to("cpu").scores > 0.8).sum())

In [ ]:
# Test with another data

test_image_path = "/content/drive/MyDrive/Colab_Notebooks/dataset/od_custom/images.jfif"
img = cv2.imread(test_image_path)
outputs = predictor(img)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
v = Visualizer(img[:, :, ::-1], metadata=MetadataCatalog.get("face_train"), scale=1)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(out.get_image()[:, :, ::-1])